In [ ]:
#REACT-NATIVE (FAST API) VERSION

In [28]:
#Unccomment the following, to install required packages if running for the first time.
#!pip install fastapi python-multipart uvicorn nest_asyncio

In [29]:
%%writefile radiant_api.py
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from transformers import BlipForConditionalGeneration, AutoProcessor
from PIL import Image
import torch
import io
import uvicorn
from pathlib import Path

# ======== CONFIG - update if your path differs ========
EXPORT_PATH = Path(r"F:\Dynoid Development\RadiantClarkiX\Chest Model\model_export")
MODEL_DIR = EXPORT_PATH / "hf_model"
PROCESSOR_DIR = EXPORT_PATH / "hf_processor"
# =====================================================

app = FastAPI(title="RadiantClarkX API")

# Allow cross-origin requests for development (Expo Snack / Expo Go)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device} (if CUDA available, will use GPU)")

# Load model and processor once (cache)
# If you get a warning about "use_fast" you can safely ignore it for now
processor = AutoProcessor.from_pretrained(str(PROCESSOR_DIR))
model = BlipForConditionalGeneration.from_pretrained(str(MODEL_DIR)).to(device)
model.eval()

@app.post("/predict")
async def predict_image(file: UploadFile = File(...)):
    """
    POST /predict
    Form-data: file -> image (jpg/png)
    Response: {"caption": "..." }
    """
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

    # Prepare inputs and generate
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_length=128, num_beams=5)
        caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return {"caption": caption}

# ONLY use uvicorn.run if you execute this file directly from terminal.
# If running in Jupyter, see instructions below (use nest_asyncio).

Writing radiant_api.py


In [ ]:
#STREAMLIT VERSION

In [34]:
%%writefile radiant.py
import streamlit as st
from pathlib import Path
from PIL import Image
import torch
from transformers import BlipForConditionalGeneration, AutoProcessor, LogitsProcessorList

# --- Config ---
st.set_page_config(
    page_title="Chest Image Captioning Demo",
    page_icon="🩺",
    layout="centered",
    initial_sidebar_state="auto",
)

# Sidebar
st.sidebar.title("📋 Instructions")
st.sidebar.write("""
1. Upload a chest image (X-ray or photo).  
2. The model will generate a caption.  
3. Use for demo purpose only — not medical advice.
""")

# Load model & processor
EXPORT_PATH = Path(r"F:\\Dynoid Development\\RadiantClarkiX\\Chest Model\\model_export")
MODEL_DIR = EXPORT_PATH / "hf_model"
PROCESSOR_DIR = EXPORT_PATH / "hf_processor"

@st.cache_resource  # caches loading so it’s fast on rerun
def load_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = BlipForConditionalGeneration.from_pretrained(str(MODEL_DIR)).to(device)
    processor = AutoProcessor.from_pretrained(str(PROCESSOR_DIR))
    return model, processor, device

model, processor, device = load_model()

# Title
st.title("🩻 Chest Image Captioning with BLIP")

# File uploader
uploaded_file = st.file_uploader("Upload a chest image", type=["jpg","jpeg","png"], accept_multiple_files=False)
if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Run inference
    with st.spinner("Generating caption…"):
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_length=128,
                output_scores=True,
                return_dict_in_generate=True
            )

        # Decode caption
        caption = processor.batch_decode(output.sequences, skip_special_tokens=True)[0]

        # Compute confidence (mean of softmax probabilities across generated tokens)
        scores = output.scores  # list of logits for each step
        probs = [torch.nn.functional.softmax(s, dim=-1) for s in scores]
        token_probs = []
        for i, token_id in enumerate(output.sequences[0][1:]):  # skip <bos>
            token_prob = probs[i][0, token_id].item()
            token_probs.append(token_prob)

        confidence = sum(token_probs) / len(token_probs) if token_probs else 0
        confidence_pct = round(confidence * 100, 2)

    # Display result
    st.markdown(
    f"""
    <h3 style='font-size: 24px; color: #2E86C1; font-weight: 700;'>
        🩺 Caption: <span style='color: white;'>{caption}</span><br>
        🔹 Confidence: <span style='color: #00FF00;'>{confidence_pct}%</span>
    </h3>
    """,
    unsafe_allow_html=True
    )

    # Badge to show done
    st.success("✅ Caption generated successfully!")

# Optional extra UI
st.markdown("---")
st.write("Demo built with Streamlit version:", st.__version__)


Overwriting radiant.py
